## Step 0 import necessary module and load data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display # Allows the use of display() for DataFrames

# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

import time
import datetime
import os

#import datacleaningp1 as dcp
import dcpv2
reload(dcpv2)

In [ ]:
# load training data
data_train_all = pd.read_csv('TrainingData_fixed_4_test_1.csv')
print 'Training data read sucessfully'
display(data_train_all.describe())

In [ ]:
# load test data
data_test_all = pd.read_csv('test_data_1.csv')
print 'Test data read successfully'

In [ ]:
data_test_backup = data_test_all.copy()
data_test_all.drop(['date'],axis = 1,inplace = True)

## Step 1 feature selection and linear regression

In [ ]:
### shuffle training data
from sklearn.utils import shuffle
data_train_all = shuffle(data_train_all,random_state = 42)

### split label from features
label = data_train_all['gaps']
data_train_all.drop(['gaps'], axis = 1, inplace= True)

### drop apparently irrelavant features
data_train_all.drop(['Weather_8','Weather_6'],axis= 1,inplace= True)
district_label = data_train_all['district_id']
data_train_all.drop(['district_id'],axis= 1,inplace= True)

### feature_preprocessing
feature_cols = data_train_all.columns
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
data_train_scaled = min_max_scaler.fit_transform(data_train_all)
data_train_scaled = pd.DataFrame(data = data_train_scaled,columns = feature_cols)
data_train_scaled['district_id'] = district_label
feature_cols = data_train_scaled.columns

### train_test_split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train_scaled, label, test_size=0.3, random_state=42)

In [ ]:
### feature selection using Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV,Lasso
from sklearn.metrics import mean_absolute_error,r2_score

reg_lasso = LassoCV(max_iter=5000)
sfm = SelectFromModel(reg_lasso, threshold='2*mean') ### feature weights lower than threshhold will be removed
sfm.fit(X_train_selected, y_train)

mask = sfm.get_support()
feature_selected = features_cols[mask]
print feature_selected

X_train2 = X_train[feature_selected]
X_test2 = X_test[featuer_selected]

### Train a ridge regression model
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV

rrm = linear_model.Ridge()
#parameters = {'alpha':np.logspace(0,2,10)}
parameters = {'alpha':np.linspace(1,10,10)}
grid_rr = GridSearchCV(rrm, parameters,scoring = 'mean_absolute_error',cv = 10)

grid_rr.fit(X_train2,y_train)

rr_final = grid_rr.best_estimator_
print rr_final.get_params()

y_pred = rr_final.predict(X_train2)
print 'r-square on training set: ',r2_score(y_train,y_pred)
print 'mae on training set: ',mean_absolute_error(y_train,y_pred)
y_pred = rr_final.predict(X_test2)
print 'r-square on test set: ',r2_score(y_test,y_pred)
print 'mae on test set: ' ,mean_absolute_error(y_test,y_pred)

In [ ]:
### measure performance on real test set
district_label_test = data_test_all['district_id']
data_test_all.drop(['district_id'],axis = 1,inplace = True)
feature_col = data_test_all.columns
data_test_scaled = min_max_scaler.transform(data_test_all)
data_test_scaled['district_id'] = district_label_test

data_test2 = data_test_scaled[feature_selected]
gaps_pred = rr_final.predict(data_test2)

data_test_backup['gaps'] = gaps_pred

import score
output_test_pred = score.output_required_format(data_test_backup)
output_test_pred.to_csv('test_pred_1.csv',index = False)
score.measure_performace()